In [1]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
from cryptography.exceptions import InvalidSignature

rsa → lets you generate RSA keys.

padding → defines how messages are padded before signing/verifying (RSA needs padding to be secure).

hashes → provides secure hash functions (here, SHA-256).

default_backend() → selects the cryptography backend (implementation details).

InvalidSignature → exception raised when verification fails.

In [2]:
# --- Generate RSA key pair ---
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)
public_key = private_key.public_key()

Generates a private RSA key (2048 bits, secure).

public_exponent=65537 is the standard choice for RSA (fast + secure).

From the private key, we derive the public key.

In [9]:
# --- Original message ---
message = b"Nature is for the satisfied or hollow/And what does it add up to in this land?/A patch of wood, some ripples in the sand./A modest hill where modest villas follow./ Give me the city streets, the urban grey,/Quays and canals that keep the water tamed,/The clouds that never look finer than when, framed/By attic windows, they go their windswept way./The least expectant have most to marvel at./Life keeps its wonders under lock and key/Until it springs them on us, rich, complete./One dreary morning all this dawned on me,/When, soaking wet in drizzly Dapper Street,/I suddenly felt happy, just like that."

b"..." means it’s stored as bytes, not a Python string.

This is important because cryptographic operations work on byte sequences.

In [10]:
def bytes_to_bitstring(data: bytes) -> str:
    return ''.join(f'{b:08b}' for b in data)

## This function takes a bytes object and turns it into a string of 0s and 1s, 
## where each byte is represented as 8 bits. 
## So: it converts bytes → string of bits (0 and 1).

## data: bytes Means that the function expects a bytes object as input (e.g., b"ABC" or bytes([65, 66, 67])).
## -> str Means that the function returns a string.

## return ''.join(f'{b:08b}' for b in data) Means that we are using a generator expression inside a ''.join(...).

## It loops over each b in the data (each b is an integer between 0 and 255, since a byte = 8 bits).

## For each b, it formats it as a binary string of length 8 using the expression: f'{b:08b}'

## b → the number (e.g., 65).

## b format code → means "binary".

## 08 → pad with zeros so the result is always 8 characters long.

## Example: 65 → "01000001".

## ''.join(...) concatenates all these 8-bit binary strings into one long string.

In [11]:
# === Step 1: Hash the message explicitly ===
digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest.update(message)
hash_bytes = digest.finalize()

print("Message:", message)
print("SHA-256 Hash (hex):", hash_bytes.hex())
print("SHA-256 Hash (bits):", bytes_to_bitstring(hash_bytes))

Message: b'Nature is for the satisfied or hollow/And what does it add up to in this land?/A patch of wood, some ripples in the sand./A modest hill where modest villas follow./ Give me the city streets, the urban grey,/Quays and canals that keep the water tamed,/The clouds that never look finer than when, framed/By attic windows, they go their windswept way./The least expectant have most to marvel at./Life keeps its wonders under lock and key/Until it springs them on us, rich, complete./One dreary morning all this dawned on me,/When, soaking wet in drizzly Dapper Street,/I suddenly felt happy, just like that.'
SHA-256 Hash (hex): e79219ac256ede772366a2d3df63ede1ed0143312a8b0a7d8d3b79dd735abefa
SHA-256 Hash (bits): 1110011110010010000110011010110000100101011011101101111001110111001000110110011010100010110100111101111101100011111011011110000111101101000000010100001100110001001010101000101100001010011111011000110100111011011110011101110101110011010110101011111011111010


In [12]:
# --- Sign the message ---
signature = private_key.sign(
    message,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

private_key.sign(...) → creates a digital signature.

PSS (Probabilistic Signature Scheme) → secure padding scheme for RSA signatures.

MGF1 (Mask Generation Function) with SHA-256 is used inside PSS.

salt_length=PSS.MAX_LENGTH → adds randomness to prevent forgery attacks.

hashes.SHA256() → the message is hashed with SHA-256 before signing (signing the hash, not the raw message).

👉 The result (signature) is a large byte string (usually same size as RSA modulus, here 256 bytes).

In [13]:
# --- Function to verify and report ---
def verify_signature(msg, sig, label="Original"):
    try:
        public_key.verify(
            sig,
            msg,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        print(f"✅ {label}: Signature is valid.")
    except InvalidSignature:
        print(f"❌ {label}: Signature is invalid!")

# --- 1. Original (valid) ---
verify_signature(message, signature, "Original")

✅ Original: Signature is valid.


public_key.verify(...) checks that:

The sig was produced with the corresponding private key.

The msg has not been altered.

The padding and hash match what was used at signing.

If verification fails, it raises InvalidSignature.

In [15]:
# --- 2. Tampered message ---
tampered_message = b"Nature is for the UNsatisfied or hollow/And what does it add up to in this land?/A patch of wood, some ripples in the sand./A modest hill where modest villas follow./ Give me the city streets, the urban grey,/Quays and canals that keep the water tamed,/The clouds that never look finer than when, framed/By attic windows, they go their windswept way./The least expectant have most to marvel at./Life keeps its wonders under lock and key/Until it springs them on us, rich, complete./One dreary morning all this dawned on me,/When, soaking wet in drizzly Dapper Street,/I suddenly felt happy, just like that."
verify_signature(tampered_message, signature, "Tampered Message")

# --- 3. Tampered signature ---
tampered_signature = signature[:-1] + b'\x00'  # corrupt last byte
verify_signature(message, tampered_signature, "Tampered Signature")

❌ Tampered Message: Signature is invalid!
❌ Tampered Signature: Signature is invalid!
